In [78]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


'C:\Users\Lee'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [79]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [80]:
users = (pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/users_interactiontea.csv'))


In [81]:
users.head(5)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,a18,-8.850000e+18,1.260000e+18,NaN,NaN,NaN
1,1465412560,VIEW,a41,-1.030000e+18,3.620000e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,a34,-1.130000e+18,2.630000e+18,NaN,NaN,NaN
3,1465413895,FOLLOW,a25,3.440000e+17,-3.170000e+18,NaN,NaN,NaN
4,1465412290,VIEW,a63,-4.450000e+17,5.610000e+18,NaN,NaN,NaN


In [82]:
event_type_strength ={
    'VIEW':1.0, 'LIKE' : 2.0, 'BOOKMARK': 2.5, 'FOLLOW' : 3.0, 'COMMENT CREATED' : 4.0   ##뷰(클릭수),좋아요,즐찾,팔로우,댓글 가중치 설정
}
users['eventStrength'] = (users.loc[:,'eventType'].apply (lambda d: event_type_strength[d]))
#interaction_df에서  eventType열을 불러와서 가중치 딕셔너리에서 뽑아서 eventStrength열을 생성후 대입.

In [83]:
users_over5 = (users
  .groupby('personId', group_keys=False) ##personId로 groupby
  .apply(lambda df: df.assign(interactCnt = lambda d: d['contentId'].nunique()))
  .loc[lambda d: d['interactCnt'] >= 5] ##상호작용, 즉 event가 발생한 것이 5회 이상인 행(유저) 만 불러옴
)


In [84]:
users_over5.loc[:,users_over5.columns == 'personId'].value_counts()

personId     
-1.030000e+18    465
-1.440000e+18    322
-2.630000e+18    244
 3.610000e+18    202
-7.090000e+17    158
                ... 
 1.230000e+18      5
 1.060000e+18      5
 1.040000e+18      5
 3.640000e+17      5
 9.200000e+18      5
Length: 397, dtype: int64

In [85]:
users_over5.loc[:,users_over5.columns == 'contentId'].value_counts()

contentId
a4           154
a46          151
a43          150
a58          148
a11          147
            ... 
a9           118
a25          116
a51          112
a21          109
a3           109
Length: 70, dtype: int64

In [86]:
users_full_df = (
  users_over5
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength'] ##한 사람이 각 콘텐츠에 행한 여러 활동들'코멘트, 클릭 등등 eventStrength의 합을 구해야함
    .sum() 
    .assign(eventScore = lambda d: np.log2(1+d['eventStrength'])) ##assign 함수로 eventScore 열 새로 할당<eventstrength에 1 더해서. 0이면 정의 안되므로 1을 더해서 보정
)##로그화 해 주는 이유는 분포를 평활하게 해 줌으로써 향후 향할 통계적 기법에 있어서 원활하게 진행하기 위함, 정규화 진행

users_full_df.head(20)

,personId,contentId,eventStrength,eventScore
0,-9.220000e+18,a1,1.0,1.000000
1,-9.220000e+18,a23,1.0,1.000000
2,-9.220000e+18,a30,1.0,1.000000
3,-9.220000e+18,a58,1.0,1.000000
4,-9.220000e+18,a66,1.0,1.000000
5,-9.170000e+18,a14,1.0,1.000000
6,-9.170000e+18,a20,1.0,1.000000
7,-9.170000e+18,a21,1.0,1.000000
8,-9.170000e+18,a23,1.0,1.000000
9,-9.170000e+18,a4,1.0,1.000000


In [87]:
from nltk.corpus import stopwords
import nltk

In [88]:
tea= (pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/teaable.csv'))
tea

,차품목,효능번호,카페인,향기준,품목,"맛,향"
0,Bengal Spice,1,X,2,a1,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,과일향(베리향)
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향"
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질, 꿀 레몬"


In [89]:
tea.loc[69,:]

차품목     요기티 Yogi Tea 피치 디톡스 티 16티백
효능번호                             4
카페인                              X
향기준                              1
품목                             a70
맛,향                          복숭아 향
Name: 69, dtype: object

In [90]:
tea

,차품목,효능번호,카페인,향기준,품목,"맛,향"
0,Bengal Spice,1,X,2,a1,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,과일향(베리향)
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향"
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질, 꿀 레몬"


In [91]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Lee Dong
[nltk_data]     woo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [92]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [93]:
users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,a18,-8.850000e+18,1.260000e+18,NaN,NaN,NaN,1.0
1,1465412560,VIEW,a41,-1.030000e+18,3.620000e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,a34,-1.130000e+18,2.630000e+18,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,a25,3.440000e+17,-3.170000e+18,NaN,NaN,NaN,3.0
4,1465412290,VIEW,a63,-4.450000e+17,5.610000e+18,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
9994,1463762672,VIEW,a4,2.940000e+18,7.230000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,1.0
9995,1463761336,VIEW,a69,-5.370000e+18,-3.330000e+18,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
9996,1463760871,VIEW,a41,8.110000e+18,-8.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
9997,1463761359,VIEW,a42,-5.370000e+18,-3.330000e+18,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0


In [94]:
scent=tea.loc[:,'맛,향']
scent

0                                                계피, 생강
1                                              과일향(베리향)
2                                           과일향(체리, 베리)
3                                          라벤더꽃향, 시트러스향
4                                          버터카라멜향, 바닐라향
                            ...                        
65                      감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추
66                                             허브 약초 생강
67    생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨...
68                                         누릅나무껍질, 꿀 레몬
69                                                복숭아 향
Name: 맛,향, Length: 70, dtype: object

In [95]:
tea_df=pd.DataFrame(tea)
tea_df

,차품목,효능번호,카페인,향기준,품목,"맛,향"
0,Bengal Spice,1,X,2,a1,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,과일향(베리향)
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향"
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초, 카다몬씨앗, 회향씨, 페퍼민트잎, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리, 감초, 유칼립투스잎, 오렌지껍질, 박하잎, 레몬그라스, 바질잎, 카아몬씨..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질, 꿀 레몬"


In [96]:
#한국어 불용어 없애기
import string
stopwords_to_remove=[',','향','(',')']
tea_df.reset_index(drop=True, inplace=True)
# Define our function to remove stopwords
def remove_stopwords(word):
    if word not in stopwords_to_remove:
        return word
    else:
        return ''
# Apply the function to our words column to clean up.


In [97]:
tea_new = (pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/teaable2.csv'))
tea_new

,차품목,효능번호,카페인,향기준,품목,"맛,향"
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향"
1,BerryBlossom White,1,O,1,a2,베리 향
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향"
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초 향, 카다몬 향, 회향씨 향, 박하 향, 생강뿌리 향, 후추 향"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리 향, 감초 향, 유칼립투스잎 향, 오렌지껍질 향, 박하 향, 레몬그라스 향..."
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질 향, 꿀 향, 레몬 향"


In [98]:
tea_new=pd.DataFrame(tea_new)
tea_new['scent_new']=tea_new['맛,향'].apply(remove_stopwords)
tea_new.head(15)

,차품목,효능번호,카페인,향기준,품목,"맛,향",scent_new
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향","계피 향, 생강 향"
1,BerryBlossom White,1,O,1,a2,베리 향,베리 향
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향","체리 향, 베리 향"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향","라벤더꽃 향, 시트러스 향"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향","버터카라멜 향, 바닐라 향"
5,Candy Cane Lane,4,△,"3,5",a6,"민트 향, 녹차 향","민트 향, 녹차 향"
6,Chai Green Tea,3,O,"2,5",a7,"계피 향, 생강 향, 녹차 향","계피 향, 생강 향, 녹차 향"
7,Chamomile Nights Herbal Tea,4,X,4,a8,캐모마일 향,캐모마일 향
8,China Green Tips,3,O,5,a9,녹차 향,녹차 향
9,Choco,3,X,2,a10,"초콜릿 향, 계피 향, 생강 향","초콜릿 향, 계피 향, 생강 향"


In [99]:
tea_new['scent_new'] = tea_new.scent_new.str.replace('[향,' ']?' , '')
tea_new


<ipython-input-99-e36ec22d48d4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  tea_new['scent_new'] = tea_new.scent_new.str.replace('[향,' ']?' , '')


,차품목,효능번호,카페인,향기준,품목,"맛,향",scent_new
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향",계피 생강
1,BerryBlossom White,1,O,1,a2,베리 향,베리
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향",체리 베리
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향",라벤더꽃 시트러스
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향",버터카라멜 바닐라
...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초 향, 카다몬 향, 회향씨 향, 박하 향, 생강뿌리 향, 후추 향",감초 카다몬 회씨 박하 생강뿌리 후추
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,허브 약초 생강
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리 향, 감초 향, 유칼립투스잎 향, 오렌지껍질 향, 박하 향, 레몬그라스 향...",생강뿌리 감초 유칼립투스잎 오렌지껍질 박하 레몬그라스 바질 카다몬 오...
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질 향, 꿀 향, 레몬 향",누릅나무껍질 꿀 레몬


In [100]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Lee Dong
[nltk_data]     woo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lee Dong woo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [101]:
#한국어 불용어 없애기
import string
stopwords_to_remove=[',','향','(',')',' ']
tea_df.reset_index(drop=True, inplace=True)
# Define our function to remove stopwords
def remove_stopwords(word):
    if word not in stopwords_to_remove:
        return word
    else:
        return ''


In [102]:
def extract_noun(words):
    token = nltk.tokenize.word_tokenize(words)
    result=[]
    for i in nltk.pos_tag(token):
        if i[1].startswith('NN'):
            result.append(i[0])

    return(', '.join(result))
tea_new['scent_words']=tea_new['scent_new'].apply(extract_noun)
tea_new

,차품목,효능번호,카페인,향기준,품목,"맛,향",scent_new,scent_words
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향",계피 생강,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,베리 향,베리,베리
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향",체리 베리,"체리, 베리"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향",라벤더꽃 시트러스,"라벤더꽃, 시트러스"
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향",버터카라멜 바닐라,"버터카라멜, 바닐라"
...,...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초 향, 카다몬 향, 회향씨 향, 박하 향, 생강뿌리 향, 후추 향",감초 카다몬 회씨 박하 생강뿌리 후추,"카다몬, 회씨, 박하, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,허브 약초 생강,"약초, 생강"
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리 향, 감초 향, 유칼립투스잎 향, 오렌지껍질 향, 박하 향, 레몬그라스 향...",생강뿌리 감초 유칼립투스잎 오렌지껍질 박하 레몬그라스 바질 카다몬 오...,"감초, 유칼립투스잎, 오렌지껍질, 박하, 레몬그라스, 바질, 카다몬, 오레가노, 파슬리"
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질 향, 꿀 향, 레몬 향",누릅나무껍질 꿀 레몬,"꿀, 레몬"


In [103]:
tea_new.drop('맛,향',axis=1)

,차품목,효능번호,카페인,향기준,품목,scent_new,scent_words
0,Bengal Spice,1,X,2,a1,계피 생강,"계피, 생강"
1,BerryBlossom White,1,O,1,a2,베리,베리
2,Black Cherry Berry,3,X,1,a3,체리 베리,"체리, 베리"
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,라벤더꽃 시트러스,"라벤더꽃, 시트러스"
4,ButterScotch Blondie,3,O,4,a5,버터카라멜 바닐라,"버터카라멜, 바닐라"
...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,감초 카다몬 회씨 박하 생강뿌리 후추,"카다몬, 회씨, 박하, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,"약초, 생강"
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,생강뿌리 감초 유칼립투스잎 오렌지껍질 박하 레몬그라스 바질 카다몬 오...,"감초, 유칼립투스잎, 오렌지껍질, 박하, 레몬그라스, 바질, 카다몬, 오레가노, 파슬리"
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,누릅나무껍질 꿀 레몬,"꿀, 레몬"


In [104]:
tea_new.drop(['scent_new','맛,향','향기준'],axis=1,inplace=True)

In [105]:
tea_new

,차품목,효능번호,카페인,품목,scent_words
0,Bengal Spice,1,X,a1,"계피, 생강"
1,BerryBlossom White,1,O,a2,베리
2,Black Cherry Berry,3,X,a3,"체리, 베리"
3,Breakfast in Paris Black Tea,1,O,a4,"라벤더꽃, 시트러스"
4,ButterScotch Blondie,3,O,a5,"버터카라멜, 바닐라"
...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,a66,"카다몬, 회씨, 박하, 생강뿌리, 후추"
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,a67,"약초, 생강"
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,a68,"감초, 유칼립투스잎, 오렌지껍질, 박하, 레몬그라스, 바질, 카다몬, 오레가노, 파슬리"
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,a69,"꿀, 레몬"


In [106]:
tea_new=tea_new.fillna('X')

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [108]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003,
    max_df=0.5,
    max_features=5000,
    stop_words=stopwords_to_remove
)

In [109]:
item_ids = tea_new['품목'].tolist()
tfidf_matrix = vectorizer.fit_transform(tea_new['scent_words'])
tfidf_feature_names = vectorizer.get_feature_names()

In [110]:
tfidf_matrix

<70x113 sparse matrix of type '<class 'numpy.float64'>'
	with 188 stored elements in Compressed Sparse Row format>

In [111]:
users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,a18,-8.850000e+18,1.260000e+18,NaN,NaN,NaN,1.0
1,1465412560,VIEW,a41,-1.030000e+18,3.620000e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,a34,-1.130000e+18,2.630000e+18,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,a25,3.440000e+17,-3.170000e+18,NaN,NaN,NaN,3.0
4,1465412290,VIEW,a63,-4.450000e+17,5.610000e+18,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
9994,1463762672,VIEW,a4,2.940000e+18,7.230000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,1.0
9995,1463761336,VIEW,a69,-5.370000e+18,-3.330000e+18,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
9996,1463760871,VIEW,a41,8.110000e+18,-8.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
9997,1463761359,VIEW,a42,-5.370000e+18,-3.330000e+18,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0


In [112]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_user_profile(person_id, interaction_indexed_df):
    interaction_person_df = interaction_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interaction_person_df['contentId'])
    
    user_item_strengths = np.array(interaction_person_df['eventStrength']).reshape(-1, 1)
    
    # 상호작용 강도를 바탕으로 가중치를 부여하여 평균 계산한다
    user_item_strengths_weighted_avg = \
        np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) /\
        np.sum(user_item_strengths)
        
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_user_profiles():
    interaction_indexed_df = (users_full_df
        .loc[lambda d: d['contentId'].isin(tea_new['품목'])]
        .set_index('personId')
    )
    user_profiles = {}
    
    for person_id in interaction_indexed_df.index.unique():
        user_profiles[person_id] = build_user_profile(person_id, interaction_indexed_df)
        
    return user_profiles

In [113]:
user_profiles = build_user_profiles()
len(user_profiles)

397

In [114]:
myprofile = user_profiles[8240000000000000000 ].flatten().tolist()
pd.DataFrame(sorted(zip(tfidf_feature_names, myprofile), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

,token,relevance
0,복숭아,0.523755
1,민트,0.368659
2,스피아민트,0.349170
3,홍차,0.273067
4,계피,0.258926
5,생강,0.179895
6,레몬,0.173785
7,베리,0.146161
8,바닐라,0.133303
9,계피 생강,0.123517


In [115]:
from sklearn.metrics.pairwise import cosine_similarity

class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, item_ids, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        # 유저 특성과 항목 특성 사이의 코사인 유사도를 구한다
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        
        # 가장 유사한 항목을 찾는다
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        # 유사도를 기준으로 유사한 항목을 정렬한다
        similar_items = sorted(
            [(item_ids[i], cosine_similarities[0, i]) for i in similar_indices],
            key=lambda x: -x[1]
        )
        
        return similar_items
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        
        # 기존에 상호작용했던 항목은 제거한다
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations = (
            pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength'])
              .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )
        
        return recommendations
content_based_model = ContentBasedRecommender(item_ids, tea_new)

In [116]:
content_based_model.recommend_items(8240000000000000000 , topn=10, verbose=False)

,contentId,recStrength
0,a60,0.523755
1,a40,0.523755
2,a55,0.523755
3,a70,0.523755
4,a50,0.368659
5,a44,0.368659
6,a41,0.368659
7,a13,0.352006
8,a64,0.349170
9,a32,0.329660


In [117]:
tea_new.tail(40)

,차품목,효능번호,카페인,품목,scent_words
30,Iced True Black,3,O,a31,홍차
31,Juniper Mint Honey,3,X,a32,"민트, 베리"
32,Lemon Lavender Lane,4,X,a33,"레몬, 라벤더"
33,Lemon Mint,4,X,a34,"레몬, 민트"
34,Lemon Strawberry,3,O,a35,"레몬, 딸기"
35,Licorice Spcie Herbal Tea,1,X,a36,"계피, 오렌지"
36,Meyer Lemon Herbal Tea,"1,4",X,a37,시트러스
37,Morning Thunder,1,O,a38,마테
38,Organic Chai,4,X,a39,"카다몸, 생강"
39,ORGANIC PEACHY GREEN,4,O,a40,복숭아


In [118]:
users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,a18,-8.850000e+18,1.260000e+18,NaN,NaN,NaN,1.0
1,1465412560,VIEW,a41,-1.030000e+18,3.620000e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,a34,-1.130000e+18,2.630000e+18,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,a25,3.440000e+17,-3.170000e+18,NaN,NaN,NaN,3.0
4,1465412290,VIEW,a63,-4.450000e+17,5.610000e+18,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
9994,1463762672,VIEW,a4,2.940000e+18,7.230000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,1.0
9995,1463761336,VIEW,a69,-5.370000e+18,-3.330000e+18,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
9996,1463760871,VIEW,a41,8.110000e+18,-8.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
9997,1463761359,VIEW,a42,-5.370000e+18,-3.330000e+18,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0


In [119]:
items_inter=users[users['personId']==8240000000000000000]
items_inter

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
15,1465415259,VIEW,a11,8.240000e+18,8.520000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
16,1465415623,VIEW,a12,8.240000e+18,8.520000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
21,1465415431,VIEW,a1,8.240000e+18,8.520000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
24,1465415565,VIEW,a40,8.240000e+18,8.520000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
47,1465415574,VIEW,a55,8.240000e+18,8.520000e+18,NaN,NaN,NaN,1.0
55,1465415430,VIEW,a59,8.240000e+18,8.520000e+18,NaN,NaN,NaN,1.0
165,1465846859,VIEW,a8,8.240000e+18,8.160000e+18,NaN,NaN,NaN,1.0
177,1465846851,VIEW,a18,8.240000e+18,8.160000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
196,1465847008,VIEW,a21,8.240000e+18,8.160000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
1267,1466022104,VIEW,a40,8.240000e+18,-8.570000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
